In [11]:
# pip install langchain
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate
!pip -q install langchain


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import DBObject
import sqlite3
import generatePrompt
from typing import List

IndentationError: expected an indented block after 'if' statement on line 13 (SchemaScraper.py, line 14)

# Loading Alpaca 7B

In [3]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch

ModuleNotFoundError: No module named 'transformers'

In [12]:
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    load_in_8bit=True,
    device_map='auto',
)

In [13]:
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain import PromptTemplate, LLMChain

template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [ ]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "What is the capital of England?"

print(llm_chain.run(question))

In [ ]:
question = "What are alpacas? and how are they different from llamas?"

print(llm_chain.run(question))

# Setting up a Chat with memory

In [ ]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [ ]:
# We are going to set the memory to go back 4 turns
window_memory = ConversationBufferWindowMemory(k=4)

In [ ]:
conversation = ConversationChain(
    llm=local_llm, 
    verbose=True, 
    memory=window_memory
)

In [ ]:
conversation.prompt.template

In [ ]:
conversation.prompt.template = '''The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 

Current conversation:
{history}
Human: {input}
AI:'''

In [ ]:
conversation.predict(input="Hi there! I am Sam")

In [ ]:
def connect_fun(database_name: str) -> DBObject:
  conn = SQLDatabase.from_uri(f"sqlite:///{database_name}?mode=ro") # open in readonly mode
  return DBObject(conn)

In [14]:

def query_fun(question: str, tables: list[str], database_connection: DBObject) -> str:
  prompt = database_connection.generate_prompt(question, tables)

  return prompt


In [15]:
# test:

connection = connect_fun("sampleDatabases/Northwind3.db")

print(
  query_fun("How many users have age over 45?", ["users"], connection)
)


TypeError: 'module' object is not callable